In [1]:
import logging, matplotlib, os, sys, glob
import scanpy as sc
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import pandas as pd
from glbase3 import genelist

In [2]:
plt.rcParams['figure.figsize']=(8,8)
sc.settings.verbosity = 3
sc.set_figure_params(dpi=200, dpi_save=200)
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['font.size']=10

In [3]:
from glbase3 import genelist, glload

In [4]:
sc.settings.figdir = 'diffexp'

In [5]:
import os
 
def mkdir(path):
 
    folder = os.path.exists(path)
 
    if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
        print("---  new folder...  ---")
        print("---  OK  ---")
 
    else:
        print("---  There is this folder!  ---")
        
gls = "/mnt/e/projects/scte-gpc/scte_ovary/gls"
mkdir(gls)

---  There is this folder!  ---


In [6]:
[os.remove(f) for f in glob.glob('{0}/*.pdf'.format(sc.settings.figdir))]
[os.remove(f) for f in glob.glob('gls/*.glb'.format(sc.settings.figdir))]
[os.remove(f) for f in glob.glob('gls/*.tsv'.format(sc.settings.figdir))]

[]

In [7]:
de_leiden = 'leiden_r0.10'

In [8]:
adata = sc.read('./de.h5ad')

In [9]:
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=True, show=False, save='genes-top25.pdf')
sc.pl.rank_genes_groups(adata, key='rank_genes_groups', show=False, save='genes.pdf')
sc.pl.rank_genes_groups_dotplot(adata, key='rank_genes_groups', show=False, save='genes-top25.pdf')

    using 'X_pca' with n_pcs = 20
Storing dendrogram info using `.uns['dendrogram_leiden_r0.10']`


{'mainplot_ax': <AxesSubplot:>,
 'group_extra_ax': <AxesSubplot:>,
 'gene_group_ax': <AxesSubplot:>,
 'size_legend_ax': <AxesSubplot:title={'center':'Fraction of cells\nin group (%)'}>,
 'color_legend_ax': <AxesSubplot:title={'center':'Mean expression\nin group'}>}

In [10]:
print(pd.DataFrame(adata.uns['rank_genes_groups']['names']))

               0           1            2        3          4
0            DCN       CRIP1          B2M    TNNI3    ZFAND2A
1           RBP1  AL928654.3         CD74  C4orf48  KCNQ1-AS1
2         RPS15A       TIMP3        HLA-B     BEX1      TUBB8
3          RPS24       ADIRF        IFI27  ATP5IF1     REC114
4          RPLP1   ADIRF-AS1          VWF   ISYNA1      PDCD5
...          ...         ...          ...      ...        ...
2995   THE1D-int  AC093908.1         DVL1  RSPH10B     TWIST1
2996   MIR3936HG     TMEM256   AC006460.2   RNF111    LTR16B1
2997        ADNP        FCMR      LTR16A2   ANXA2R     ARRDC1
2998  MER101-int       NEDD9  HERVK13-int    AP3S2      DUSP7
2999      SPATS2       NAA20       ARFIP1   SPOCK1       AMFR

[3000 rows x 5 columns]


In [11]:
topall = pd.DataFrame(adata.uns['rank_genes_groups']['names']) # get all;
fcs = pd.DataFrame(adata.uns['rank_genes_groups']['logfoldchanges'])
padj = pd.DataFrame(adata.uns['rank_genes_groups']['pvals_adj'])

In [12]:
# Matrix of DE genes:

groups = list(topall.columns.values)

newcols = {}

for group in groups:
    newcols[group] = []

    t = zip([i[group] for i in adata.uns['rank_genes_groups']['names']], [i[group] for i in adata.uns['rank_genes_groups']['logfoldchanges']], [i[group] for i in adata.uns['rank_genes_groups']['pvals_adj']])

    print('Group: {0}'.format(group))
    print(t)

    for item in t:
        if item[1] < 2: # fold change
            continue
        if item[2] > 0.01: # just in case
            continue

        newcols[group].append({'name': item[0], 'log2FC': item[1], 'q': item[2]})

Group: 0
Group: 1
Group: 2
Group: 3
Group: 4


In [13]:
# join all and draw a dotplot:
for group in newcols:
    print(newcols[group])
    if newcols[group]:
        gl = genelist() 
        gl.load_list(newcols[group])
        gl.saveTSV('gls/de_genes-grp{0}.tsv'.format(group))
        gl.save('gls/de_genes-grp{0}.glb'.format(group))

        genes = [i['name'] for i in newcols[group]]
        #sc.pl.dotplot(adata, genes, groupby=de_leiden, dot_max=0.7, dendrogram=True, standard_scale='var', show=False, save='de-grp{0}.pdf'.format(group))
        #sc.pl.matrixplot(adata, genes, groupby=de_leiden, dendrogram=True, standard_scale='var', show=False, save='de-grp{0}.pdf'.format(group))

INFO    : Saved 'gls/de_genes-grp0.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp0.glb'
INFO    : Saved 'gls/de_genes-grp1.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp1.glb'
INFO    : Saved 'gls/de_genes-grp2.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp2.glb'
INFO    : Saved 'gls/de_genes-grp3.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp3.glb'
INFO    : Saved 'gls/de_genes-grp4.tsv'
INFO    : Saved binary version of list: 'gls/de_genes-grp4.glb'


[{'name': 'DCN', 'log2FC': 2.9755795, 'q': 0.0}, {'name': 'RBP1', 'log2FC': 2.2938578, 'q': 0.0}, {'name': 'FHL2', 'log2FC': 2.287563, 'q': 0.0}, {'name': 'APOE', 'log2FC': 2.569924, 'q': 0.0}, {'name': 'SERPINF1', 'log2FC': 2.5113304, 'q': 0.0}, {'name': 'C1S', 'log2FC': 2.4651191, 'q': 0.0}, {'name': 'STAR', 'log2FC': 2.5050414, 'q': 0.0}, {'name': 'ASH2L', 'log2FC': 2.2124999, 'q': 0.0}, {'name': 'OGN', 'log2FC': 2.7462704, 'q': 0.0}, {'name': 'CPZ', 'log2FC': 2.5355551, 'q': 0.0}, {'name': 'GPR78', 'log2FC': 2.548953, 'q': 0.0}, {'name': 'SERPINE2', 'log2FC': 2.2581086, 'q': 0.0}, {'name': 'OLFML3', 'log2FC': 2.4711828, 'q': 0.0}, {'name': 'CLDN11', 'log2FC': 2.2300143, 'q': 0.0}, {'name': 'PDGFRA', 'log2FC': 2.225016, 'q': 0.0}, {'name': 'GSTM5', 'log2FC': 2.1199744, 'q': 0.0}, {'name': 'KRT19', 'log2FC': 2.1102612, 'q': 0.0}, {'name': 'MMP23B', 'log2FC': 2.1195097, 'q': 0.0}, {'name': 'KLHDC8A', 'log2FC': 2.3507652, 'q': 0.0}, {'name': 'BASP1', 'log2FC': 2.483938, 'q': 0.0}, {'na

In [14]:
for grp in newcols:
    if not newcols[grp]:
        continue
    for k in newcols[grp]:
        title = k['name']
        sc.pl.umap(adata, color=k['name'], size=20, legend_loc='on data',
            title=title,
            vmin=0, vmax=3,
            show=False, save='-markers-grp{0}-{1}.pdf'.format(grp, k['name']))
        #sc.pl.violin(adata, [k], groupby='disease', size=0, log=False, cut=0, show=False, save='markers-{0}-disease.pdf'.format(k))
        #sc.pl.violin(adata, [k], groupby='cell_type', size=0, log=False, cut=0, show=False, save='markers-{0}-cell_type.pdf'.format(k))